Read Toronto Postal Code Data

In [1]:
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] # do not run this unattended (not guaranteed)

Read HTML table into *panda* dataframe

In [2]:
import pandas as pd

#The dataframe will consist of three columns: Postcode, Borough, and Neighbourhood
df = pd.read_html(str(table))[0]

df.shape

(289, 3)

Ignore cells with a borough that is Not assigned

In [3]:

#Ignore cells with a borough that is Not assigned
df = df[df.Borough != 'Not assigned']

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [4]:
import numpy as np
df['Neighbourhood'] = np.where(df['Neighbourhood'] == 'Not assigned', df['Borough'], df['Neighbourhood'])

Group Neighborhoods by Postal code

In [13]:
dff=df.groupby(['Postcode','Borough']).Neighbourhood.agg([('Neighbourhood', ', '.join)]).reset_index()

Geographical coordinates of each postal code

In [6]:
dfg=pd.read_csv('Geospatial_Coordinates.csv')

Combine postal code and co-ordinates data

In [16]:
dfm=pd.merge(dff, dfg, on='Postcode')
dfm.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Show map

In [22]:
import folium
tor_latitude =  43.70011
tor_longitude= -79.4163

map_tor = folium.Map(location=[tor_latitude, tor_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(dfm['Latitude'], dfm['Longitude'], dfm['Borough'], dfm['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)
    
map_tor.save('test2.html')

